In [ ]:
import os
import pandas as pd
import requests

# Read the CSV
csv_path = 'Path to csv file having META Data'
df = pd.read_csv(csv_path, skiprows=1)  # Skips the first row

# Initialize the base URL
base_url = "https://dr18.sdss.org/sas/dr18/prior-surveys/sdss4-dr17-eboss/photoObj/frames/{rerun}/{run}/{camcol}/frame-{band}-{run_padded}-{camcol}-{field_padded}.fits.bz2"

# Initialize a set to keep track of unique URLs
unique_urls = set()

# Populate the set with unique URLs
for _, row in df.iterrows():
    run_padded = f"{row['run']:06d}"
    field_padded = f"{row['field']:04d}"
    
    for band in ['u','g','r','i','z']:
        url = base_url.format(
            rerun=row['rerun'],
            run=row['run'],
            camcol=row['camcol'],
            band=band,
            run_padded=run_padded,
            field_padded=field_padded
        )
        unique_urls.add(url)

# Display the count of unique URLs and the count of FITS files to download
print(f"There are {len(df)} objid in the CSV.")
print(f"There are {len(unique_urls)} unique FITS files to download in total.")

# Ensure the folder exists
if not os.path.exists('Download_Folder'):
    os.makedirs('Download_Folder')

# Download the FITS files
for url in unique_urls:
    # Extract the filename from the URL
    file_name = url.split('/')[-1]
    
    # Download the FITS file
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join('Download_Folder', file_name), 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {file_name}. Status code: {response.status_code}")
